In [1]:
import pandas as pd
import torch

df = pd.read_csv("../data/lichess_db_standard_rated_2014-12.csv")
df.head()


,white,black,result
0,carlsen89,goltrpoat,0.0
1,nunutsu,sergeblanc,0.0
2,Boss1,piongu,0.0
3,Melchisedech,nabov,1.0
4,bumblebee,Exodus_z,1.0


In [2]:
# Node list
node_list = pd.concat([df['white'], df['black']]).unique().tolist()
num_node = len(node_list)
node_dict = {player_id: node for node, player_id in enumerate(node_list)}

In [3]:
# Initialize edge dictionary
edge_dict = {}

# Iterate over dataframe rows
for idx, row in df.iterrows():
    white = node_dict[row['white']]
    black = node_dict[row['black']]
    edge = (white, black)
    
    if edge not in edge_dict:
        edge_dict[edge] = {"win": 0, "lose": 0, "draw": 0}
    
    if row['result'] == 1:
        edge_dict[edge]["win"] += 1
    elif row['result'] == 0:
        edge_dict[edge]["lose"] += 1
    elif row['result'] == 0.5:
        edge_dict[edge]["draw"] += 1

In [4]:
# Create edge_index tensor
edge_index = torch.tensor([[white,black] for (white, black), _ in edge_dict.items()], dtype=torch.long).t().contiguous()
# Create edge_attr tensor
edge_attr = torch.tensor([[stats['win'], stats['draw'], stats['lose']] for _, stats in edge_dict.items()], dtype=torch.float)

In [5]:
print(edge_index.shape)
print(edge_attr.shape)

torch.Size([2, 978358])
torch.Size([978358, 3])


In [6]:
torch.save({'edge_index': edge_index, 'edge_attr': edge_attr, 'num_node':num_node}, '../data/graph_data.pt')

In [ ]:
df["white"] = node_dict[df["white"]]
df["black"] = node_dict[df["black"]]
df.head()